## Eye color classification

In [14]:
train_data_directory = 'dataset/train/eyes'
test_data_directory = 'dataset/test/eyes'

import cv2 # open cv
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

load_model = tf.keras.models.load_model
image = tf.keras.preprocessing.image
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
Sequential = tf.keras.models.Sequential
Dense = tf.keras.layers.Dense
Activation = tf.keras.layers.Activation
Flatten = tf.keras.layers.Flatten
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D

picture_width = 400
picture_height = 540

CLASSES = ['black','blue','brown']
MODEL_NAME = 'eyes'

In [16]:
# initiate data preprocessing tools

# This step processes the images into a format that
# 1. makes the data readable to the model
# 2. provides more training material for the model to train from
# the `training_data_processor` below scales the data so that it can be
# a model input, but also takes each image and augments it so that
# the model can learn from multiple variations of the same image.
# it flips it horizontally, rotates it, shifts it, and more so that 
# the model learns from the soil photo rather than the orientation size
training_data_processor  = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    zoom_range = 0.2,
    rotation_range = 10,
    shear_range = 0.2,
    height_shift_range = 0.1,
    width_shift_range = 0.1,
)
# for the testing images, we don't need to create multiple variatinos
test_data_processor = ImageDataGenerator(rescale=1./255)


# load data into python

training_data = training_data_processor.flow_from_directory(
    train_data_directory,
    target_size = (picture_width, picture_height), # pixels
    batch_size = 32,
    class_mode = 'categorical',
)

testing_data = test_data_processor.flow_from_directory(
    test_data_directory,
    target_size = (picture_width, picture_height),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = False
)

Found 74 images belonging to 3 classes.
Found 29 images belonging to 3 classes.


### Model building

In [17]:
# choose model parameters
num_conv_layers = 3
num_dense_layers = 2
layer_size = 29 # limit the layer size as we dont want to over fit the model
num_training_epochs = 25

model = Sequential()

# begin adding properties to model variable
# e.g. add a convulutional layer
model.add(Conv2D(layer_size, (3,3), input_shape=(picture_width, picture_height, 3)))
model.add(Activation('relu')) # rectified linear unit
model.add(MaxPooling2D(pool_size=(2,2)))

# add additional convolutional layers based on num_conv_layers
for _ in range(num_conv_layers-1):
    model.add(Conv2D(layer_size, (3,3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

# reduce dimensionality
model.add(Flatten())

# add fully connected "dense" layers if specified
for _ in range(num_dense_layers):
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# output layer
model.add(Dense(len(CLASSES)))
model.add(Activation('softmax'))

# compile the sequential model with all added properties
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

# use the data already loaded to train/tune the model
model.fit(
    training_data, 
    epochs = num_training_epochs, 
    validation_data = testing_data)

# save the trained model
model.save(f'{MODEL_NAME}.h5')

Epoch 1/20
3/3 [==============================] - 11s 3s/step - loss: 1.3142 - accuracy: 0.2838 - val_loss: 1.1409 - val_accuracy: 0.3448
Epoch 2/20
3/3 [==============================] - 9s 4s/step - loss: 1.1282 - accuracy: 0.3784 - val_loss: 1.1077 - val_accuracy: 0.3448
Epoch 3/20
3/3 [==============================] - 11s 3s/step - loss: 1.0932 - accuracy: 0.3784 - val_loss: 1.0947 - val_accuracy: 0.3448
Epoch 4/20
3/3 [==============================] - 11s 3s/step - loss: 1.0644 - accuracy: 0.4189 - val_loss: 1.0527 - val_accuracy: 0.4828
Epoch 5/20
3/3 [==============================] - 11s 5s/step - loss: 1.0377 - accuracy: 0.5000 - val_loss: 1.0035 - val_accuracy: 0.4828
Epoch 6/20
3/3 [==============================] - 12s 3s/step - loss: 1.0101 - accuracy: 0.4324 - val_loss: 0.9544 - val_accuracy: 0.6207
Epoch 7/20
3/3 [==============================] - 10s 3s/step - loss: 0.9918 - accuracy: 0.5270 - val_loss: 1.3155 - val_accuracy: 0.3793
Epoch 8/20
3/3 [===================

In [18]:
model.evaluate(training_data)

3/3 [==============================] - 3s 897ms/step - loss: 0.6411 - accuracy: 0.7162


[0.6411091685295105, 0.7162162065505981]